In [1]:
import numpy as np
import tensorflow as tf
import os
import pickle
import numpy.random as rnd
from sklearn.utils import shuffle

/home/ec2-user/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


## Load in data

In [2]:
import pickle
import boto3
from io import BytesIO

In [3]:
s3 = boto3.resource('s3')
data_subsets = ['train', 'val']
data = {}

for name in data_subsets:
    with BytesIO() as files:
        path = "omniglot_images/" +name+ ".pickle"
        s3.Bucket("research-paper-omniglot-data").download_fileobj(path, files)
        files.seek(0)    # move back to the beginning after writing
        (X,c) = pickle.load(files)
        data[name] = X

path = '../omniglot_images/'
data_subsets = ["train", "val", "test"]

data = {}
categories = {}
info = {}
        
for name in data_subsets:
    file_path = os.path.join(path, name + ".pickle")
    print("loading data from {}".format(file_path))
    with open(file_path,"rb") as f:
        (X,c) = pickle.load(f)
        data[name] = X
        categories[name] = c

In [4]:
def create_train_data(size, s='train'):
    #get train data and shape
    X=data[s]
    n_classes, n_examples, w, h = X.shape
    
    #initialize 2 empty arrays for the input size in a list
    pairs=[np.zeros((size, h, w,1)) for i in range(2)]
    
    #initialize vector for the targets
    targets=np.zeros((size,1))
    
    for x in range(size):
        #randomly sample one class (character)
        category = rnd.choice(n_classes,1,replace=False)
        #randomly sample one example from class (1-20 characters)
        idx_1 = rnd.randint(0, n_examples)
        pairs[0][x,:,:,:] = X[category, idx_1].reshape(w, h, 1)
        #randomly sample again one example from class and add last class with modulo
        # ..to ensure not same class pairs are created
        idx_2 = (idx_1 + rnd.randint(0, n_examples)) % n_examples
        #pick images of different class for 1st half and same class for 2nd half
        if x >= size // 2:
            category_2 = category
            targets[x] = 1
        else: 
        #add a random number to the category modulo n classes to ensure 2nd image has
        # ..different category
            idx_2 = rnd.randint(0, n_examples) 
            category_2 = (category + rnd.randint(1,n_classes)) % n_classes
            targets[x] = 0
        pairs[1][x,:,:,:] = X[category_2,idx_2].reshape(w, h,1)
        
    return pairs, targets

In [5]:
train_set, train_labels = create_train_data(10000)
val_set, val_labels = create_train_data(3000)

## Functions

In [6]:
def loguniform(low=0, high=1, size=None, base=np.exp(1)):
    return(np.power(base, np.random.uniform(np.log(low), np.log(high), size)))

In [7]:
def uniform_sample(low=0, high=1, size=None):
    return(np.random.uniform(low, high, size))

In [8]:
#returns random integer from discrete uniform distribution with closed interval [low,high]
def random_integer(low=0, high=10, size=None):
    return(np.random.randint(low, high + 1, size))

In [9]:
def categorical_random(input_vector, size=None):
    return(np.random.choice(input_vector))

## Squash function

In [10]:
def squash(input_vector, axis):
    normalised_input = tf.reduce_sum(tf.square(input_vector), axis = axis, keepdims = True)
    scale = tf.divide(normalised_input, tf.add(normalised_input, 1.))
    vector = tf.divide(input_vector, tf.sqrt(tf.add(normalised_input, epsilon)))
    output = tf.multiply(scale, vector)
    
    return(output)

## Convolutional layer

In [11]:
def convolutional(input_data, conv_shape, stride_shape, name, relu=True):
    weights = tf.get_variable('W'+name, initializer=tf.truncated_normal(conv_shape, stddev=0.3))
    bias = tf.get_variable('B'+name, initializer=tf.truncated_normal([conv_shape[-1]], stddev=0.3))
    out_layer = tf.nn.conv2d(input_data, weights, stride_shape, padding = 'VALID')
    out_layer_bias = tf.add(out_layer, bias)
    
    if relu == True:
        out_layer_final = tf.nn.relu(out_layer_bias)
        return(out_layer_final)
    
    return(out_layer_bias)

## Primarycaps

In [12]:
def primarycaps(input_data, conv_shape, stride_shape, primaryCaps_channels, caps1_size, caps2_size, pose_size, batch):
    output = convolutional(input_data, conv_shape, stride_shape, relu=False, name='primaryCaps')
    filter_size = output.get_shape().as_list()[1]
    caps1_raw = tf.reshape(output, [-1,filter_size*filter_size*primaryCaps_channels,caps1_size], name='caps1_raw')
    caps1_output = squash(caps1_raw, axis=-1)
    caps1_output_expand = tf.expand_dims(caps1_output, axis=-1)
    caps1_output_expand2 = tf.expand_dims(caps1_output_expand, axis=2)
    caps1_output_expand2_tiled = tf.tile(caps1_output_expand2, [1,1,caps2_size,1,1], name = 'caps1_out_tiled')
    
    weight_matrix = tf.get_variable('Weight_matrix', initializer=tf.truncated_normal([filter_size*filter_size*primaryCaps_channels, caps2_size, pose_size, caps1_size], stddev=0.1))
    #weight_matrix_tiled = tf.tile(weight_matrix, [batch, 1, 1, 1, 1], name = 'W_matrix_tiled')
    #caps2_predicted = tf.matmul(weight_matrix_tiled, caps1_output_expand2_tiled, name='caps2_predicted')
    caps2_predicted = tf.einsum('abdc,iabcf->iabdf', weight_matrix, caps1_output_expand2_tiled)
    
    return(caps2_predicted)

## Routing by agreement

In [13]:
def routing_by_agreement(input_data, caps2_size, rounds, batch):
    raw_weights = tf.zeros([batch, input_data.get_shape().as_list()[1], caps2_size, 1, 1], name = 'raw_weights')
    
    for i in range(rounds):
        routing_weights = tf.nn.softmax(raw_weights, axis=2, name = 'routing_weights' + str(i))
        weighted_predictions = tf.multiply(routing_weights, input_data, name = 'weighted_predictions' + str(i))
        weighted_sum = tf.reduce_sum(weighted_predictions, axis=1, name = 'weighted_sum' + str(i), keepdims = True)
        caps2_output = squash(weighted_sum, axis=-2)
    
        #caps2_output_tiled = tf.tile(caps2_output, [1, input_data.get_shape().as_list()[1], 1, 1, 1], name = 'caps2_output_tiled'+ str(i))
        #agreement = tf.matmul(input_data, caps2_output_tiled, transpose_a = True, name = 'agreement'+ str(i))
        agreement = tf.einsum('iabcd,ifbcd->iabcd', input_data, caps2_output)
        raw_weights = tf.add(raw_weights, agreement, name = 'raw_weights' + str(i))
        
    return(caps2_output)

In [14]:
def create_dense_layer(input_data, input_shape, neurons, name):
    weights = tf.get_variable(name+'_W', initializer=tf.truncated_normal([input_shape, neurons], stddev=0.2))
    bias = tf.get_variable(name+'b', initializer=tf.truncated_normal([neurons], mean=0.5, stddev=0.02))
    fully_connected = tf.add(tf.matmul(input_data, weights), bias)
    out_dense_activation = tf.nn.sigmoid(fully_connected)
    
    return(out_dense_activation)

## Graph

In [15]:
def create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X,fully_layer_size,stride_conv1,stride_conv2):
    conv1 = convolutional(X, [conv1_filter,conv1_filter,X.get_shape().as_list()[-1],
                                  conv1_channels],[1,stride_conv1,stride_conv1,1], name='conv')
    primary = primarycaps(conv1, [primaryCaps_filter,primaryCaps_filter,conv1.get_shape().as_list()[-1],
                              primaryCaps_channels*caps1_size], [1,stride_conv2,stride_conv2,1],primaryCaps_channels,
                      caps1_size,caps2_size, pred_matrix_size, batch=tf.shape(X)[0])
    output = routing_by_agreement(primary, caps2_size, routing_rounds, batch=tf.shape(X)[0])
    flat = tf.reshape(output, [-1, caps2_size*pred_matrix_size])
    fully_connected = create_dense_layer(flat, caps2_size*pred_matrix_size , fully_layer_size, 'fully')
    
    return(fully_connected)

In [16]:
def create_graph(learning_rate,beta1,beta2,caps1_size,caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,fully_layer_size,stride_conv1,stride_conv2):
    X1 = tf.placeholder(tf.float32, [None, 105, 105, 1])
    X2 = tf.placeholder(tf.float32, [None, 105, 105, 1])
    y = tf.placeholder(tf.float32, [None, 1])

    with tf.variable_scope('siamese') as scope:
        output1 = create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X1,fully_layer_size,stride_conv1,stride_conv2)
        scope.reuse_variables()
        output2 = create_network(caps1_size, caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,X2,fully_layer_size,stride_conv1,stride_conv2)

    l1_dis = tf.abs(tf.subtract(output1, output2))

    weights = tf.Variable(tf.truncated_normal([fully_layer_size, 1], stddev=0.03), name='w_final')
    bias = tf.Variable(tf.truncated_normal([1], stddev=0.01), name='b_final')
    fully_final = tf.add(tf.matmul(l1_dis, weights), bias)
    y_estimate = tf.nn.sigmoid(fully_final)

    cross_entropy = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(labels = y, logits = fully_final))
    optimiser = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1=beta1, beta2=beta2).minimize(cross_entropy)
    
    correct_prediction = tf.equal(tf.round(y_estimate), y)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))
    
    return(optimiser, cross_entropy, accuracy, X1, X2, y)

## Tensor checker

In [17]:
def check_size(batch_size,conv1_filter,stride_conv1,primaryCaps_filter,stride_conv2,primaryCaps_channels,caps1_size,caps2_size,pred_matrix_size):
    conv1_output_filter = np.floor((105-conv1_filter)/stride_conv1 + 1)
    conv2_output_filter = np.floor((conv1_output_filter - primaryCaps_filter)/stride_conv2 + 1)
    size_tensor = batch_size*conv2_output_filter*conv2_output_filter*primaryCaps_channels*caps2_size*pred_matrix_size*caps1_size
    size_tensor2 = conv2_output_filter*conv2_output_filter*primaryCaps_channels
    print('tensor1:',size_tensor)
    print('tensor2:',size_tensor2)
    if (size_tensor <= 900000000) & (size_tensor2 <= 4096):
        return(True)
    else:
        return(False)

## Setup training

In [18]:
learning_rate = 0.1
beta1 = 0.999
beta2 = 0.999
caps1_size = 4
caps2_size = 16
pred_matrix_size = 16
conv1_channels = 16
conv1_filter = 3
primaryCaps_channels = 16
primaryCaps_filter = 3
routing_rounds = 1
fully_layer_size = 16
stride_conv1 = 1
stride_conv2 = 1
batch_size = 32

settings = {'iteration' : 0,
            'validation_accuracy' : 0,
            'learning_rate' : 0.1,
           'beta1' : 0.999,
           'beta2' : 0.999,
           'caps1_size' : 4,
           'caps2_size' : 16,
           'pred_matrix_size' : 16,
           'conv1_channels' : 16,
           'conv1_filter' : 3,
           'primaryCaps_channels' : 16,
           'primaryCaps_filter' : 3,
           'routing_rounds' : 1,
           'fully_layer_size' : 16,
           'stride_conv1' : 1,
           'stride_conv2' : 1,
           'batch_size' : 32}

In [19]:
random_search_iterations = 3
epochs = 10
epsilon = 1e-7
best_acc_val = 0

#If run on AWS
config = tf.ConfigProto(allow_soft_placement=True)
config.gpu_options.allocator_type = 'BFC'
config.gpu_options.allow_growth=True


for i in range(random_search_iterations):
    
    size_tensor = False
    
    while not size_tensor:
        learning_rate = loguniform(1e-5, 0.1)
        beta1 = uniform_sample(0.,1.)
        beta2 = uniform_sample(0.,1.)
        caps1_size = categorical_random([8])
        caps2_size = categorical_random([16,32,48,64])
        pred_matrix_size = categorical_random([16])
        conv1_channels = categorical_random([16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256])
        conv1_filter = random_integer(3,35)
        primaryCaps_channels = categorical_random([16,32,48,64])
        primaryCaps_filter = random_integer(3,20)
        routing_rounds = random_integer(1,5)
        fully_layer_size = categorical_random([16,32,48,64,80,96,112,128,144,160,176,192,208,224,240,256])
        stride_conv1 = random_integer(1,3)
        stride_conv2 = random_integer(1,3)
        batch_size = categorical_random([64])
        size_tensor = check_size(batch_size,conv1_filter,stride_conv1,primaryCaps_filter,stride_conv2,primaryCaps_channels,caps1_size,caps2_size,pred_matrix_size)
        
    optimiser, cross_entropy, accuracy, X1, X2, y = create_graph(learning_rate,beta1,beta2,caps1_size,caps2_size,pred_matrix_size,conv1_channels,conv1_filter,primaryCaps_channels,primaryCaps_filter,routing_rounds,fully_layer_size,stride_conv1,stride_conv2)
    init_op = tf.global_variables_initializer()
    
    with tf.Session(config=config) as sess:
        init_op.run()

        total_batch = int(10000/batch_size)
        total_batch_val = int(3000/batch_size)

        for epoch in range(epochs):
            avg_cost = 0
            acc = 0
            for i in range(total_batch):
                batch_x1, batch_x2, batch_y = shuffle(train_set[0],train_set[1], train_labels, n_samples = batch_size)
                a, c, accur= sess.run([optimiser, cross_entropy, accuracy], feed_dict={X1: batch_x1, X2: batch_x2, y: batch_y})
                avg_cost += c/total_batch
                acc += accur
            print('epoch:', epoch)
            print('avg_cost:', avg_cost)
            print('train_acc:', acc/total_batch)
            acc_val = 0
            for iterations in range(total_batch_val):
                batch_x1, batch_x2, batch_y = shuffle(val_set[0],val_set[1], val_labels, n_samples = batch_size)
                val_acc = sess.run(accuracy, feed_dict={X1: batch_x1, X2: batch_x2, y: batch_y})
                acc_val += val_acc
            validation_accuracy = acc_val/total_batch_val
            print('val_acc:', validation_accuracy)
            
        if validation_accuracy > best_acc_val:
            best_acc_val = validation_accuracy
            settings['iteration'] = i
            settings['validation_accuracy'] = best_acc_val
            settings['learning_rate'] = learning_rate
            settings['beta1'] = beta1
            settings['beta2'] = beta2
            settings['caps1_size'] = caps1_size
            settings['caps2_size'] = caps2_size
            settings['pred_matrix_size'] = pred_matrix_size
            settings['conv1_channels'] = conv1_channels
            settings['conv1_filter'] = conv1_filter
            settings['primaryCaps_channels'] = primaryCaps_channels
            settings['primaryCaps_filter'] = primaryCaps_filter
            settings['routing_rounds'] = routing_rounds
            settings['fully_layer_size'] = fully_layer_size
            settings['stride_conv1'] = stride_conv1
            settings['stride_conv2'] = stride_conv2
            settings['batch_size'] = batch_size
            f = open("../settings" + str(i) +".txt","w")
            f.write( str(settings) )
            f.close() 
            
    tf.reset_default_graph()

tensor1: 28311552.0
tensor2: 576.0
epoch: 0
avg_cost: 0.693281058890697
train_acc: 0.491786858974359
val_acc: 0.48219086021505375
epoch: 1
avg_cost: 0.6932252876651588
train_acc: 0.49589342948717946
val_acc: 0.5157930107526881
epoch: 2
avg_cost: 0.693117543481863
train_acc: 0.5038060897435898
val_acc: 0.5023521505376344
epoch: 3
avg_cost: 0.6931143504304759
train_acc: 0.5040064102564102
val_acc: 0.48823924731182794
epoch: 4
avg_cost: 0.6931457957014059
train_acc: 0.5017027243589743
val_acc: 0.501008064516129
epoch: 5
avg_cost: 0.6931045666719091
train_acc: 0.504707532051282
val_acc: 0.48185483870967744
epoch: 6
avg_cost: 0.6931476724835544
train_acc: 0.5015024038461539
val_acc: 0.5060483870967742
epoch: 7
avg_cost: 0.693168065295769
train_acc: 0.5
val_acc: 0.4946236559139785
epoch: 8
avg_cost: 0.6931443556379054
train_acc: 0.5017027243589743
val_acc: 0.5047043010752689
epoch: 9
avg_cost: 0.6932387976692276
train_acc: 0.4946915064102564
val_acc: 0.4801747311827957
tensor1: 1207959552.0


In [23]:
tf.reset_default_graph()

In [20]:
settings

{'validation_accuracy': 0.7248641304347826,
 'learning_rate': 0.0004750834954300537,
 'beta1': 0.5604519304186675,
 'beta2': 0.3609095332268254,
 'caps1_size': 8,
 'caps2_size': 64,
 'pred_matrix_size': 4,
 'conv1_channels': 240,
 'conv1_filter': 3,
 'primaryCaps_channels': 48,
 'primaryCaps_filter': 10,
 'routing_rounds': 3,
 'fully_layer_size': 16,
 'stride_conv1': 4,
 'stride_conv2': 2,
 'batch_size': 64}